# Keras API - Tensorflow v2

Keras is the high level API for TensorFlow
 
 - If you're an engineer, Keras provides you with reusable blocks such as layers, metrics, training loops, to support common use cases. It provides a high-level user experience that's accessible and productive
 - If you're a researcher, you may prefer not to use these built-in blocks such as layers and training loops, and instead create your own. Of course, Keras allows you to do this. In this case, Keras provides you with templates for the blocks you write, it provides you with structure, with an API standard for things like Layers and Metrics. This structure makes your code easy to share with others and easy to integrate in production workflows
 - The same is true for library developers: TensorFlow is a large ecosystem. It has many different libraries. In order for different libraries to be able to talk to each other and share components, they need to follow an API standard. That's what Keras provides

Crucially, Keras brings high-level UX and low-level flexibility together fluently: you no longer have on one hand, a high-level API that's easy to use but inflexible, and on the other hand a low-level API that's flexible but only approachable by experts. Instead, you have a spectrum of workflows, from the very high-level to the very low-level. Workflows that are all compatible because they're built on top of the same concepts and objects.

### The base Layer class

All layers are pretty much derived from this class only. A Layer will encapsulate a state and computation. State being weights/bias and computation being forward pass (defined under call method)

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Layer

class Linear(Layer):
    
    "Implementation of y = w.x + b"

    def __init__(self, units = 32, input_dim = 32):
        super(Linear, self).__init__()
        
        # Initializing the weight initializing scheme - Random Normal Distribution
        w_init = tf.random_normal_initializer()
        
        # Declaring the Weight Matrix using the scheme, we give the shape (dimensions), no of units 
        # and specify whether we can update them or not using the trainable Flag
        self.w = tf.Variable(initial_value = w_init(shape = (input_dim, units), dtype = 'float32'), trainable = True)
        
        # Initializing the bias initializing scheme - Zeros
        b_init = tf.zeros_initializer()
        
        # Declaring the Bias Matrix using the scheme, we give the shape (dimensions) 
        # and specify whether we can update them or not using the trainable Flag 
        self.b = tf.Variable(initial_value = b_init(shape = (units,), dtype = 'float32'), trainable = True)
    
    # Forward pass function. This function is called whenever the forward pass is happening
    def call(self, inputs):
        
        # Defining the Matrix Multiplication operation - w.x + b
        return tf.matmul(inputs, self.w) + self.b


In [2]:
# Instantiating the layer with 4 units with an input layer dimension of 2
linear_layer = Linear(4, 2)

In [3]:
# Calling the function 
y = linear_layer(tf.ones((2, 2)))
assert y.shape == (2, 4)

### The Layer class takes care of tracking the weights assigned to it as attributes

In [4]:
# Weights are automatically tracked under the weights property
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

### Another way of initializing the weights

Instead of doing this way,
 - w_init = tf.random_normal_initializer()
 - self.w = tf.Variable(initial_value = w_init(shape = shape, dtype = 'float32'))

We can do it shortly in this way,
 - self.w = self.add_weight(shape = shape, initializer = 'random_normal')

### We can create the Layer in a lazy way and eliminate the input dimension from the constructor. We can have a build function for specifically initializing the weights with a specific dimension.

Here the build function gets called automatically

In [5]:
class Linear(Layer):
    
    "Implementation of y = w.x + b"
    
    def __init__(self, units = 32):
        super(Linear, self).__init__()
        self.units = units

    def build(self, input_shape):
        
        # Initialization of weights with the given input shape
        self.w = self.add_weight(shape = (input_shape[-1], self.units), initializer = 'random_normal', trainable = True)
        
        # Initialization of bias with the given input shape
        self.b = self.add_weight(shape = (self.units,), initializer = 'random_normal', trainable = True)

    def call(self, inputs):
        
        # Forward pass function. This function is called whenever the forward pass is happening 
        return tf.matmul(inputs, self.w) + self.b
    

In [6]:
# Instantiate the Layer
linear_layer = Linear(4)

In [7]:
# This will also call build(input_shape) and create the weights
y = linear_layer(tf.ones((2, 2)))
assert len(linear_layer.weights) == 2

### Trainable & Non-Trainable Weights

We use the trainable flag to freeze and unfreeze the weights in a layer

In [8]:
class ComputeSum(Layer):
    
    """Returns the sum of the inputs."""

    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        
        # Create a non-trainable weight
        self.total = tf.Variable(initial_value = tf.zeros((input_dim,)), trainable = False)

    def call(self, inputs):
        
        # Add the inputs and compute the sum
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total  


In [9]:
# Initialize the Layer
my_sum = ComputeSum(2)
x = tf.ones((2, 2))

y = my_sum(x)
print(y.numpy())  

y = my_sum(x)
print(y.numpy())  

assert my_sum.weights == [my_sum.total]
assert my_sum.non_trainable_weights == [my_sum.total]
assert my_sum.trainable_weights == []

[2. 2.]
[4. 4.]


### Composing multiple Layers into bigger computation block of layers

Layers can be recursively nested to create bigger computation blocks. Each layer will track the weights of its sublayers (both trainable and non-trainable)

In [10]:
class MLP(Layer):
    
    """Simple stack of Linear layers."""

    def __init__(self):
        super(MLP, self).__init__()
        
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(10)

    # Forward Pass function    
    def call(self, inputs):
        
        # Input Layer with ReLU activation
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        
        # Hidden Layer with ReLU activation
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)


In [11]:
# Initializing the block
block = MLP()

# The first call to the block object will create the weights
y = block(tf.ones(shape=(3, 64)))

# Weights are recursively tracked
assert len(block.weights) == 6

### Defining a model using Keras Sub-Classing technique

In [12]:
#Keras Custom-Class
class MyModel(tf.keras.Model):
    def __init__(self, num_classes = 10):
        super(MyModel, self).__init__()
   
    #Define your layers here
        inputs = tf.keras.Input(shape = (28, 28))  
        self.l1 = tf.keras.layers.Flatten()
        self.l2 = tf.keras.layers.Dense(512, activation = 'relu', name = 'dense1')
        self.l3 = tf.keras.layers.Dropout(0.2)
        self.final = tf.keras.layers.Dense(10, activation = tf.nn.softmax, name = 'output1')
    
    def call(self, inputs):
    #Define the forward pass
        x = self.l1(inputs)
        x = self.l2(x)
        x = self.l3(x)
        return self.final(x)

In [13]:
#Instantiate the model
model = MyModel()
model.compile(optimizer= tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#model.summary()
#model.fit(x, y)

### Defining a model using Keras Functional API

In [14]:
#Using Keras Functional API
inputs = tf.keras.Input(shape = (28,28))  
x = tf.keras.layers.Flatten()(inputs)
x = tf.keras.layers.Dense(512, activation='relu', name = 'dense2')(x)
x = tf.keras.layers.Dropout(0.2)(x)
final = tf.keras.layers.Dense(10, activation = tf.nn.softmax, name = 'output2')(x)

In [15]:
#Instantiate the model
model_1 = tf.keras.Model(inputs = inputs, outputs = final)
model_1.compile(optimizer= tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#model_1.summary()
#model_1.fit(x, y)

### Defining a model using Sequential : Method 1

In [16]:
#Using Sequential 
model_2 = tf.keras.models.Sequential([
 tf.keras.layers.Flatten(),
 tf.keras.layers.Dense(512, activation = tf.nn.relu, name = 'dense2'),
 tf.keras.layers.Dropout(0.2),
 tf.keras.layers.Dense(10, activation = tf.nn.softmax, name = 'output2')
])
model_2.compile(optimizer= tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#model_2.summary()
#model_2.fit(x, y)

### Defining a model using Sequential : Method 2

In [17]:
#Using Sequential
model_3 = tf.keras.models.Sequential()
model_3.add(tf.keras.layers.Flatten())
model_3.add(tf.keras.layers.Dense(512, activation='relu', name = 'dense3'))
model_3.add(tf.keras.layers.Dropout(0.2))
model_3.add(tf.keras.layers.Dense(10,activation=tf.nn.softmax, name = 'output3'))
model_3.compile(optimizer= tf.keras.optimizers.Adam(), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
#model_3.summary()
#model_3.fit(x, y)